Dependencies

In [1]:
import SimpleITK as sitk
import torch 
import numpy as np
from models import unet

Load Model

In [3]:
model = unet.UNet(in_channels=1,num_classes=19, start_filts= 32, up_mode='upsample')
#setting cpu for laptop inf testing
model.load_state_dict(torch.load("C:/Users/Konstantinos/Desktop/Spider Inference Models Test/model_20240408_174231_7", map_location=torch.device('cpu')))

model.eval()

c:\Github Repositories\LabML Unet\spider-seg-e19005\models\unet.py:213: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
  init.xavier_normal(m.weight)
c:\Github Repositories\LabML Unet\spider-seg-e19005\models\unet.py:214: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  init.constant(m.bias, 0)


UNet(
  (conv_final): Conv2d(32, 19, kernel_size=(1, 1), stride=(1, 1))
  (down_convs): ModuleList(
    (0): DownConv(
      (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (1): DownConv(
      (conv1): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (2): DownConv(
      (conv1): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (3): DownConv(
      (conv1): Conv2d(128, 256, kernel_size=(3, 3),

"Import" unique masks and one-hot encoding from training 

In [4]:
unique_masks_a = np.array([0. ,   1. ,  2.   ,3.  , 4. ,  5. ,  6. ,  7. ,  8.,   9. ,100. ,201. ,202. ,203. ,204., 205. ,206. ,207. ,208. ,209.])

val_range = np.arange(20)
print(val_range)
key_range = unique_masks_a #no of unique values in labels, including BG 
print(key_range)

mapping_dict = dict(zip(key_range, val_range))

# Define a function to perform the mapping using the dictionary
def map_to_specified_set(value):
    return mapping_dict.get(value, np.nan)  # Return np.nan for values not found in the mapping_dict

# Create a vectorized version of the mapping function
value_map = np.vectorize(map_to_specified_set)

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]
[  0.   1.   2.   3.   4.   5.   6.   7.   8.   9. 100. 201. 202. 203.
 204. 205. 206. 207. 208. 209.]


Test In MRI from images removed from training sets  

In [5]:
in_mha = sitk.ReadImage("out_inference/132_t1.mha")

in_arr = sitk.GetArrayFromImage(in_mha) #3d arr

if(in_arr.shape[0] > in_arr.shape[1] or  in_arr.shape[0] > in_arr.shape[2]):
    in_arr = np.transpose(in_arr, (2, 0, 1))

print(in_arr.shape)

#image normalisation
for idx in range (in_arr.shape[0]):
    #print(in_arr[idx, :, :].shape)
    idx_slice = in_arr[idx, : , :]
    if (idx == 0):
        image_max = np.max(idx_slice)
        image_min = np.min(idx_slice)
    else:
        if(np.max(idx_slice) > image_max):
            image_max = np.max(idx_slice)
        if(np.min(idx_slice) < image_min):
            image_min = np.min(idx_slice)

print("max", image_max, "min", image_min)

out_arr = np.empty_like(in_arr)

#convert each slice to tensor run inf on each slice and save in new np arr
for idx in range(in_arr.shape[0]):
    in_slice = in_arr[idx, :, :]
    in_tensor = torch.from_numpy(in_slice)
    in_tensor = (in_tensor - image_min) / (image_max - image_min)
    in_tensor = in_tensor.to(torch.float32)
    in_tensor = in_tensor.unsqueeze(0)
    in_tensor = in_tensor.unsqueeze(0)
    in_tensor.to('cpu')
    print(in_tensor.shape) 
    out_tensor = model(in_tensor) #OK 
    print("out tensor dims", out_tensor.shape)

     #TODO parse one hot dict here to undo one hot encoding 
    #out_tensor_indices = torch.argmax(out_tensor, dim = 1)

    #print("out tensor undo one hot shape", out_tensor_indices.shape)

    out_tensor_arr = out_tensor.detach().numpy()

    out_indices_arr = np.argmax(out_tensor_arr, axis = 1)
    out_values_arr = np.vectorize(lambda index: unique_masks_a[index])(out_indices_arr)
    print("out indices arr shape", out_indices_arr.shape)



    out_arr = out_values_arr[0, :, :]

    print("out values array unique", np.unique(out_arr))
    print("array going into sitk", out_arr.shape)

    out_image = sitk.GetImageFromArray(out_arr)
    
    sitk.WriteImage(out_image, "C:/Users/Konstantinos/Desktop/sitk inference out/{}.mha".format(idx))
    
    break #testing for one image 
    
    



    


(15, 320, 320)
max 3096 min -1000
torch.Size([1, 1, 320, 320])
out tensor dims torch.Size([1, 19, 320, 320])
out indices arr shape (1, 320, 320)
out values array unique [  0.   1.   2.   3.   4.   5.   6.   9. 100. 201. 202. 203.]
array going into sitk (320, 320)


Split the image into slices in loop and run inference model eval 